In [63]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from main import init_env_vars
from langchain import FAISS
from langchain.embeddings import OpenAIEmbeddings

init_env_vars()

# db = FAISS.load_local('index', OpenAIEmbeddings())
db = FAISS.load_local('temp_index/main/merged', OpenAIEmbeddings())
FAISS.merge_from(db, FAISS.load_local('temp_index/help/merged', OpenAIEmbeddings()))
retriever = db.as_retriever(search_type='similarity', search_kwargs={'k': 4})


def ask_bot(question, ai_instruction):
    system_template = ai_instruction + "\n\n{summaries}"
    messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
    chain_type_kwargs = {"prompt": ChatPromptTemplate.from_messages(messages)}
    qa = RetrievalQAWithSourcesChain.from_chain_type(
        llm=ChatOpenAI(temperature=0),
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs)
    result = qa({"question": question})
    return result

In [64]:
def get_sources(result):
    source_urls = []
    for document in result['source_documents']:
        url = document.metadata['source'].replace("website/", "https://")
        if url not in source_urls:
            source_urls.append(url)

    return source_urls

In [72]:
ai_instruction = "Give your answer in a bullet point format to be used for slide presentations"
# ai_instruction = ""
question = "How Zorin OS handle system upgrades?"

response = ask_bot(question, ai_instruction)

print(response['answer'])
print(get_sources(response))

Zorin OS handles system upgrades through its built-in Software Updater tool. This tool checks for updates to the operating system and installed software packages, and allows users to easily download and install them with just a few clicks. Zorin OS also provides long-term support (LTS) releases, which receive security and stability updates for up to 5 years, ensuring that users can continue to use their system with confidence. Additionally, Zorin OS offers an easy direct upgrade method from Zorin OS Core to Pro, allowing users to upgrade while keeping their files, apps, and settings.
['https://zorin.com/os/pro.html', 'https://zorin.com/os/pro/index.html', 'https://help.zorin.com/index.html', 'https://zorin.com/os.html']


Here are some image links that you can use in your presentation:

1. Zorin OS Education homepage: https://zorin.com/os/education/
2. Zorin OS Education screenshot: https://zorin.com/wp-content/uploads/2021/08/zorin-os-education-screenshot.png
3. Zorin OS Education Lite screenshot: https://zorin.com/wp-content/uploads/2021/08/zorin-os-education-lite-screenshot.png
4. Zorin OS Education apps: https://zorin.com/wp-content/uploads/2021/08/zorin-os-education-apps.png
5. Zorin OS Education classroom control: https://zorin.com/wp-content/uploads/2021/08/zorin-os-education-classroom-control.png
6. Zorin OS Education Veyon management console: https://zorin.com/wp-content/uploads/2021/08/zorin-os-education-veyon-management-console.png
7. Zorin OS Education Lite on an old computer: https://zorin.com/wp-content/uploads/2021/08/zorin-os-education-lite-old-computer.png
8. Zorin OS Education logo: https://zorin.com/wp-content/uploads/2021/08/zorin-os-education-logo.png

Make sure to check the usage rights of each image before using them in your presentation.
['https://zorin.com/os/education/index.html', 'https://zorin.com/about.html', 'https://zorin.com/about/index.html']
